In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from scipy.stats import pearsonr
from statsmodels.stats.multitest import multipletests

# Generate random RDMs
np.random.seed(42)

num_neural_rdms = 4
num_dnn_rdms = 7

rdm_size = 1870

neural_rdms = [np.random.rand(rdm_size, rdm_size) for _ in range(num_neural_rdms)]

dnn_rdms = [np.random.rand(rdm_size, rdm_size) for _ in range(num_dnn_rdms)]

# Apply PCA to reduce dimensionality
reduced_neural_rdms = []
for neural_rdm in neural_rdms:
    scaler = StandardScaler()
    neural_rdm_scaled = scaler.fit_transform(neural_rdm)

    pca_model = PCA(n_components=100)
    pca_model.fit(neural_rdm_scaled)

    reduced_neural_rdm = pca_model.transform(neural_rdm_scaled)
    reduced_neural_rdms.append(reduced_neural_rdm)

reduced_dnn_rdms = []
for dnn_rdm in dnn_rdms:
    scaler = StandardScaler()
    dnn_rdm_scaled = scaler.fit_transform(dnn_rdm)

    pca_model = PCA(n_components=100)
    pca_model.fit(dnn_rdm_scaled)

    reduced_dnn_rdm = pca_model.transform(dnn_rdm_scaled)
    reduced_dnn_rdms.append(reduced_dnn_rdm)

# Correlation between reduced neural and DNN RDMs
correlations = np.zeros((num_neural_rdms, num_dnn_rdms))
p_values = []

for i in range(num_neural_rdms):
    for j in range(num_dnn_rdms):
        reduced_neural_i = reduced_neural_rdms[i]
        reduced_dnn_j = reduced_dnn_rdms[j]

        # Calculate Pearson correlation coefficient
        correlation, p_value = pearsonr(reduced_neural_i.flatten(), reduced_dnn_j.flatten())
        correlations[i, j] = correlation
        p_values.append(p_value)

print("\nCorrelation matrix between reduced Neural and DNN RDMs:\n", correlations)
print("\np-values for Pairwise Hypothesis Test:\n", p_values)

# Visualize the correlation matrix with heatmap
sns.heatmap(correlations, annot=True, cmap="coolwarm", fmt=".2f", vmin=-1, vmax=1)
plt.title("Correlation Matrix between Reduced Neural and DNN RDMs")
plt.show()

# Multiple testing correction using Benjamini-Hochberg procedure
reject, corrected_p_values, _, _ = multipletests(p_values, method='fdr_bh')

print("\nResults for Pairwise Hypothesis Test (Benjamini-Hochberg Correction):\n")
for i in range(num_neural_rdms):
    for j in range(num_dnn_rdms):
        correlation = correlations[i, j]
        p_value = corrected_p_values[i * num_dnn_rdms + j]
        is_rejected = reject[i * num_dnn_rdms + j]

        print(f"NN layer {i + 1} vs. DNN layer {j + 1}: correlation={correlation:.3f}, p-value={p_value:.3f}, rejected={is_rejected}")
